# <img src="https://upload.wikimedia.org/wikipedia/commons/6/60/NISAR_artist_concept.jpg" width=400 align="left"/>
<img src="https://upload.wikimedia.org/wikipedia/commons/9/9b/NISAR_Mission_Logo.png" width=400 align="left"/><br><br><br><br><br>



# Convert ALOS2 Data to GCOV

In preparation for NISAR, this notebook converts ALOS-2 SLC data to NISAR RSLC to NISAR GCOV.


These steps can be run in the command line, details are below.

In [ ]:
import os
import sys
from pathlib import Path
import subprocess
import fnmatch
import zipfile
import h5py
from string import Template
from osgeo import gdal
import numpy as np
import math
import boto3
import shutil
import matplotlib.pyplot as plt
import imageio


## Choose your place to save your outputs. 

The home directory should be used (not scratch) unless very large storage space is needed. Files will be moved to the S3 bucket after processing. 

In [ ]:
notebook_dir = Path(os.getcwd())
working_dir = Path('/scratch/alex_eco_test/')


In [ ]:
bucket_name = "nisar-st-data-ondemand"
# s3_path = 'alex/ALOS2_zip/'
s3_path = 'ALOS2_zip/'
s3 = boto3.client("s3")
response = s3.list_objects_v2(
            Bucket=bucket_name,
            Prefix = s3_path)
            # MaxKeys=100)
            # Delimiter = '/')
contents = response.get('Contents')
existing_ALOS2_zipfiles = [contents[i].get('Key').split('/')[-1] for i in range(len(contents)) if '.zip' in contents[i].get('Key')]
existing_ALOS2_zippaths = [contents[i].get('Key') for i in range(len(contents)) if '.zip' in contents[i].get('Key')]


## Choose your AOI

New directories will be made to store files for this AOI

In [ ]:
aoi = 'TALL'
aoi_str = aoi.replace(" ", "_")


aoi_dir = working_dir/aoi_str
RSLC_dir = aoi_dir / 'RSLC'
ALOS2_dir = aoi_dir / 'ALOS2'
DEM_dir = aoi_dir / 'DEM'
GCOV_dir = aoi_dir / 'GCOV'
TMP_dir = aoi_dir / 'TMP'

Path(aoi_dir).mkdir(parents=True, exist_ok=True)
Path(RSLC_dir).mkdir(parents=True, exist_ok=True)
Path(ALOS2_dir).mkdir(parents=True, exist_ok=True)
Path(DEM_dir).mkdir(parents=True, exist_ok=True)
Path(GCOV_dir).mkdir(parents=True, exist_ok=True)
Path(TMP_dir).mkdir(parents=True, exist_ok=True)



## Get the ALOS2 URLs 

If you install **pandas** and **openpyxl** to your isce3_src package, you can search the ALOS2 spreadsheed directly. Otherwise, you need to copy the list of URLS manually



In [ ]:
import pandas as pd
xlsx = pd.ExcelFile('/scratch/alex_eco_test/ALOS-2_Restricted_Available_at_ASF_20231227.xlsx')


Filter the ALOS2 data using name, path, frame, etc

In [ ]:
# df_filtered = df[(df['Frame']==580) & (df['Path']==51)]
sheet = 'Forest (L1.1)'
df = pd.read_excel(xlsx,sheet)
df_filtered = df[df['Request']==aoi]
list_of_ALOS_SLCs = list(df_filtered['Zip Link'])
list_of_ALOS_SLCs.sort()
list_of_ALOS_SLCs

Or manually enter the list of URLs to download

In [ ]:
# ## Example using Wax Lake Delta
# list_of_ALOS_SLCs = [
#                     'https://cumulus.asf.alaska.edu/L1.1/A4/0000436588_001001_ALOS2390510580-210816.zip',
#                     'https://cumulus.asf.alaska.edu/L1.1/A4/0000470239_001001_ALOS2427770580-220425.zip',
#                     'https://cumulus.asf.alaska.edu/L1.1/A4/0000475404_001001_ALOS2442260580-220801.zip',
#                     'https://cumulus.asf.alaska.edu/L1.1/A4/0000477810_001001_ALOS2444330580-220815.zip',
#                     'https://cumulus.asf.alaska.edu/L1.1/A4/0000479846_001001_ALOS2446400580-220829.zip',
#                     'https://cumulus.asf.alaska.edu/L1.1/A4/0000509061_001001_ALOS2479520580-230410.zip',
#                     'https://cumulus.asf.alaska.edu/L1.1/A4/0000510737_001001_ALOS2481590580-230424.zip',
#                     'https://cumulus.asf.alaska.edu/L1.1/A4/0000526924_001001_ALOS2498150580-230814.zip',
#                     'https://cumulus.asf.alaska.edu/L1.1/A4/0000528515_001001_ALOS2500220580-230828.zip',
                    
#                     ]

## Download the ALOS2 files using wget

In [ ]:
ALOS2_ids = []
response = s3.list_objects_v2(
            Bucket=bucket_name,
            Prefix = s3_path)
            # MaxKeys=100)
            # Delimiter = '/')
contents = response.get('Contents')
existing_ALOS2_zipfiles = [contents[i].get('Key').split('/')[-1] for i in range(len(contents)) if '.zip' in contents[i].get('Key')]
existing_ALOS2_zippaths = [contents[i].get('Key') for i in range(len(contents)) if '.zip' in contents[i].get('Key')]

for file in list_of_ALOS_SLCs[:]:
    filename =  file.split('/')[-1]
    ALOS2_ids.append(filename)
    print('Requested File: ', filename)
    print(os.path.isdir(ALOS2_dir/filename.split('/')[-1][:-4]))
    print(os.path.isfile(ALOS2_dir/filename))
    print((filename in existing_ALOS2_zipfiles))
    if os.path.isdir(ALOS2_dir/filename.split('/')[-1][:-4])==True & (filename in existing_ALOS2_zipfiles)==True:
        print('ALOS2 is stored on S3 and already available locally and unzipped')
    elif (os.path.isfile(ALOS2_dir/filename)==False) & (filename in existing_ALOS2_zipfiles):
        i = existing_ALOS2_zipfiles.index(filename)
        s3_path_new = existing_ALOS2_zippaths[i]
        print('\tALOS2 zip is already available at S3 PATH: ', s3_path_new)
        s3.download_file(bucket_name, s3_path_new , filename)
        print('\tMove ALOS2 zip from S3 to local')
        shutil.move(notebook_dir/filename, ALOS2_dir/filename)
        
    elif (os.path.isfile(ALOS2_dir/filename)==False) & (filename not in existing_ALOS2_zipfiles):
        print('\tALOS2 is not available anywhere')
        print('\tDownloading ALOS2 zip ')
        command = f"wget -P {ALOS2_dir} -q {file}"
        output = subprocess.check_output(command, shell=True)
        print('\tMoving a copy ALOS2 zip to S3 bucket')
        s3.upload_file(Filename= str(ALOS2_dir / filename), Bucket=bucket_name, Key='%s%s/%s' %(s3_path,aoi_str,filename))
    
    elif (os.path.isfile(ALOS2_dir/filename)==True) & (filename not in existing_ALOS2_zipfiles):
        print('\tALOS2 is available locally, but not on S3')
        print('\tMoving a copy ALOS2 zip to S3 bucket')
        s3.upload_file(Filename= str(ALOS2_dir / filename), Bucket=bucket_name, Key='%s%s/%s' %(s3_path,aoi_str,filename))

    else: 
        print('\tALOS2 zip file exists locally and on S3')
        
    if os.path.isdir(ALOS2_dir/filename.split('/')[-1][:-4])==False:
        print('\tunzipping the ALOS2 file locally')
        try:
            with zipfile.ZipFile(ALOS2_dir/filename, 'r') as zip_ref:
                zip_ref.extractall(ALOS2_dir/filename.split('/')[-1][:-4])
        except:
            print('\tALOS2 zip is bad, re-downloading')
            os.remove(ALOS2_dir/filename)
            command = f"wget -P {ALOS2_dir} -q {file}"
            output = subprocess.check_output(command, shell=True)
            print('\t\tMoving a copy ALOS2 zip to S3 bucket')
            s3.upload_file(Filename= str(ALOS2_dir / filename), Bucket=bucket_name, Key='%s%s/%s' %(s3_path,aoi_str,filename))
            with zipfile.ZipFile(ALOS2_dir/filename, 'r') as zip_ref:
                zip_ref.extractall(ALOS2_dir/filename.split('/')[-1][:-4])
        print('\t\tdeleting the local zip file')
        # os.remove(ALOS2_dir/filename) ## comment to remove the alos2 zip file from your local workspace
    

## Get a list of folders for each ALOS2 file

In [ ]:
ALOS2folders = [os.path.join(dirpath,f)
                for dirpath,dirnames, files in os.walk(ALOS2_dir)
                for f in fnmatch.filter(dirnames,'*')]
ALOS2folders.sort()
ALOS2folders

## Convert ALOS2 SLC to NISAR RSLC format

In [ ]:
for ALOS2folder in ALOS2folders[:]:
    ALOS2_id = ALOS2folder.split('/')[-1]
    print('')
    print(ALOS2_id)
    
    ##ALOS2 SLC --> NISAR RSLC
    command = f"conda run -n isce3_src /home/jovyan/isce3/share/nisar/examples/alos2_to_nisar_l1.py -i {ALOS2folder} -o {RSLC_dir/ALOS2_id}.h5"
  
    if os.path.isfile(f"{RSLC_dir/ALOS2_id}.h5")==True:
        print('NISAR RSLC already converted')
    else:
            
        try: os.remove(f"{RSLC_dir/ALOS2_id}.h5")
        except:''
        output = subprocess.check_output(command, shell=True)    
        print('')
        print(command)  
        

In [ ]:
ALOS2_RSLCs = [os.path.join(dirpath,f)
                for dirpath,dirnames, files in os.walk(RSLC_dir)
                for f in fnmatch.filter(files,'*')]
ALOS2_RSLCs.sort()
ALOS2_RSLCs



## Get a DEM for the first ALOS2 file in the stack 

In [ ]:

ALOS2_ref = ALOS2_RSLCs[0].split('/')[-1][:-3]
print('')
print(ALOS2_ref)

## Get NISAR DEM
command = f"conda run -n isce3_src /home/jovyan/isce3/python/packages/nisar/workflows/stage_dem.py -p {RSLC_dir/ALOS2_ref}.h5 -o {DEM_dir/ALOS2_ref}.vrt"
   

if os.path.isfile(f"{DEM_dir/ALOS2_ref}.vrt")==False:
    print('')
    print(command) 
    output = subprocess.check_output(command, shell=True)

## Process NISAR RSLC to GCOV

### If you want to change to UTM coordiates, set utm = True and choose the x and y posting. The default is 20m

In [ ]:
x_posting = ''
y_posting = ''
EPSG = ''

utm = True
if utm:
    src = gdal.Open(f"{DEM_dir/ALOS2_ref}.vrt")
    ulx, xres, xskew, uly, yskew, yres  = src.GetGeoTransform()
    lrx = ulx + (src.RasterXSize * xres)
    lry = uly + (src.RasterYSize * yres)

    x1,y1,x2,y2 = math.floor(ulx),math.floor(uly),math.floor(lrx),math.floor(lry)
    zone = int(np.ceil((ulx + 180)/6))

    if y1>=0:
        EPSG = 32600+zone
    elif y1<0:
        EPSG = 32600+zone

    x_posting = 20
    y_posting = 20
    
print(EPSG)



## Run the GCOV processor on the first/reference image in the stack. 
This image will be used to determine the bounding box of the remaining images in the stack

In [ ]:
filein = open('gcov_template.yaml')
template = Template(filein.read())
replacements = {'inputfile':  f"{RSLC_dir/ALOS2_ref}.h5",
                'outputfile':  f"{GCOV_dir/ALOS2_ref}_gcov_%s.h5" %(EPSG),
                'demfile': f"{DEM_dir/ALOS2_ref}.vrt",
                'tmp': f"{TMP_dir}/",
                'epsg': EPSG,
                'xposting': x_posting,
                'yposting': y_posting,
                'top_left_y': '',
                'top_left_x' : '',
                'bottom_right_y':'',
                'bottom_right_x':'',
                'y_snap':'',
                'x_snap':''
                }
makeoutput = template.substitute(replacements)
file = open('%s/%s.yaml' %(GCOV_dir,ALOS2_ref),'w')
file.write(makeoutput)
file.close()
filein.close()

command = f"conda run -n isce3_src /home/jovyan/isce3/python/packages/nisar/workflows/gcov.py {GCOV_dir/ALOS2_ref}.yaml"

if os.path.isfile(f"{GCOV_dir/ALOS2_ref}_gcov_utm.h5")==False:
    print('')
    print(command)
    output = subprocess.check_output(command, shell=True)


## Get top left coordinates of bottom right coordinates of reference image

In [ ]:
f = h5py.File(f"{GCOV_dir/ALOS2_ref}_gcov_%s.h5" %(EPSG), "r") 
a_group_key = list(f.keys())[0]
ds_x = f[a_group_key]['LSAR']['GCOV']['grids']['frequencyA']['xCoordinates'][()]      # returns as a h5py dataset object
ds_y = f[a_group_key]['LSAR']['GCOV']['grids']['frequencyA']['yCoordinates'][()]      # returns as a h5py dataset object

ulx = x_posting * round(ds_x[0]/x_posting)
lrx = x_posting * round(ds_x[-1]/x_posting)
uly = y_posting * round(ds_y[0]/y_posting)
lry = y_posting * round(ds_y[-1]/y_posting)

print('Force top left to be: %s %s' %(ulx,uly))
print('Force top right to be: %s %s' %(lrx,lry))



## Process the remaining RSLC files to GCOV using the bounding box set by the reference images

In [ ]:
for ALOS2_RSLC in ALOS2_RSLCs[:]:
    ALOS2_id = ALOS2_RSLC.split('/')[-1][:-3]
    print('')
    print(ALOS2_id)

    filein = open('gcov_template.yaml')
    template = Template(filein.read())
    replacements = {'inputfile':  f"{RSLC_dir/ALOS2_id}.h5",
                    'outputfile':  f"{GCOV_dir/ALOS2_id}_gcov_%s.h5" %(EPSG),
                    'demfile': f"{DEM_dir/ALOS2_ref}.vrt",
                    'tmp': f"{TMP_dir}/",
                    'epsg': EPSG,
                    'xposting': x_posting,
                    'yposting': y_posting,
                    'top_left_y': uly,
                    'top_left_x':ulx,
                    'bottom_right_y':lry,
                    'bottom_right_x': lrx,
                    'y_snap':'',#y_posting,
                    'x_snap':''#x_posting
                    }
    makeoutput = template.substitute(replacements)
    file = open('%s/%s.yaml' %(GCOV_dir,ALOS2_id),'w')
    file.write(makeoutput)
    file.close()
    filein.close()

    command = f"conda run -n isce3_src /home/jovyan/isce3/python/packages/nisar/workflows/gcov.py {GCOV_dir/ALOS2_id}.yaml"
    print('')
    print(command)
    if os.path.isfile(f"{GCOV_dir/ALOS2_id}_gcov_utm.h5")==False:
        output = subprocess.check_output(command, shell=True)


## See final list of NISAR GCOV data files

In [ ]:
NISAR_GCOVs = [os.path.join(dirpath,f)
                for dirpath,dirnames, files in os.walk(GCOV_dir)
                for f in fnmatch.filter(files,'*.h5')]
NISAR_GCOVs.sort()
NISAR_GCOVs

## Check that all H5 files have the same dimensions and extent
## Make Figures

In [ ]:

for NISAR_GCOV in NISAR_GCOVs[:]:
    ALOS2_id = NISAR_GCOV.split('/')[-1][:-3]
    print('')
    print(ALOS2_id)
    f = h5py.File(f"{GCOV_dir/ALOS2_id}.h5", "r") 
    a_group_key = list(f.keys())[0]
    ds_x = f[a_group_key]['LSAR']['GCOV']['grids']['frequencyA']['xCoordinates'][()]      # returns as a h5py dataset object
    ds_y = f[a_group_key]['LSAR']['GCOV']['grids']['frequencyA']['yCoordinates'][()]      # returns as a h5py dataset object
    HH = f[a_group_key]['LSAR']['GCOV']['grids']['frequencyA']['HHHH'][()] 
    HV = f[a_group_key]['LSAR']['GCOV']['grids']['frequencyA']['HVHV'][()] 
    print('Width x Height:' , ds_x.shape, ds_y.shape)
    ulx = x_posting * round(ds_x[0]/x_posting)
    lrx = x_posting * round(ds_x[-1]/x_posting)
    uly = y_posting * round(ds_y[0]/y_posting)
    lry = y_posting * round(ds_y[-1]/y_posting)
    extent= (ulx, lry, lrx, uly) 
    print('Top Left: ', ulx,uly)
    print('Bottom Right: ', lrx,lry)
    
    fig, [ax1,ax2] = plt.subplots(1,2,figsize = (15,10))
    im1 = ax1.imshow(HH, interpolation = 'nearest',
          extent = extent, vmin=0,vmax=0.5,cmap='viridis')
    cbar1 = plt.colorbar(im1, ax=ax1,
            shrink=0.5, orientation='horizontal',
            pad=0.1, aspect=50,label='HH')
    im2 = ax2.imshow(HV, interpolation = 'nearest',
          extent = extent, vmin=0,vmax=0.1,cmap='viridis')
    cbar2 = plt.colorbar(im2, ax=ax2,
            shrink=0.5, orientation='horizontal',
            pad=0.1, aspect=50,label='HV')
    plt.suptitle(ALOS2_id)
    plt.savefig(GCOV_dir/ (ALOS2_id + '_HH-HV.png'))


## Make an animation

In [ ]:
all_figs = [os.path.join(dirpath,f)
                for dirpath,dirnames, files in os.walk(GCOV_dir)
                for f in fnmatch.filter(files,'*.png')]
all_figs.sort()
with imageio.get_writer('%s/%s.gif' %(GCOV_dir,aoi_str),mode='I',duration=300) as writer:
    for file in all_figs:
        print(file)
        image = imageio.imread(file)
        writer.append_data(image)
        

## Clean TMP files

In [ ]:
shutil.rmtree(TMP_dir) 
shutil.rmtree(ALOS2_dir)

## Move files to S3 Bucket


In [ ]:
# folders = [os.path.join(dirpath,f)
#                 for dirpath,dirnames, files in os.walk(aoi_dir)
#                 for f in fnmatch.filter(dirnames,'*')]
# for folder in folders:
#     files = [os.path.join(dirpath,f)
#                 for dirpath,dirnames, files in os.walk(folder)
#                 for f in fnmatch.filter(files,'*')]
#     # print(folder)
#     parent = folder.split('/')[-1]
#     print(parent)
#     for file in files:
#         print(file)
#         os.system('aws s3 mv %s s3://nisar-st-data-ondemand/ALOS2_processed/%s/%s/%s' %(file,aoi_str,parent,file))


In [ ]:
## Command line option

######## ALOS2 SLC to NISAR RSLC
# for f in /scratch/alex_eco_test/ALOS2/southfork/*/; 
# do python /home/jovyan/isce3/share/nisar/examples/alos2_to_nisar_l1.py -i $f -o "${f%.*}".h5; 
# done

######## Get NISAR DEM
# for f in /scratch/alex_eco_test/ALOS2/southfork/rslc/*; 
# do python /home/jovyan/isce3/python/packages/nisar/workflows/stage_dem.py -p $f -o "${f%.*}".vrt; 
# done

######## NISAR RSLC to NISAR GCOV
# for f in /scratch/alex_eco_test/ALOS2/southfork/rslc/*.yaml; 
# do python /home/jovyan/isce3/python/packages/nisar/workflows/gcov.py $f; 
# done